# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bredasdorp,-34.5322,20.0403,16.49,75,66,1.80,ZA,1704670502
1,1,weno,7.4515,151.8468,28.35,69,89,9.35,FM,1704670502
2,2,ushuaia,-54.8000,-68.3000,9.81,53,40,7.72,AR,1704670166
3,3,newala kisimani,-10.9500,39.2833,20.80,90,100,1.22,TZ,1704670503
4,4,west island,-12.1568,96.8225,26.99,83,75,7.20,CC,1704670503


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [12]:
%%capture --no-display

# Configure the map plot
map_plot= city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City"
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [17]:
# Narrow down cities that fit criteria and drop any results with null values
city_ideal_weather_df = city_data_df[(city_data_df['Max Temp']<27) & (city_data_df['Max Temp']>21) & (city_data_df['Wind Speed']<4.5) & (city_data_df['Cloudiness']==0)]

# Drop any rows with null values
city_ideal_weather_df = city_ideal_weather_df.dropna()

# Display sample data
city_ideal_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
116,116,kailua-kona,19.6406,-155.9956,26.98,69,0,3.09,US,1704414015
164,164,san lorenzo,-25.3333,-57.5333,26.52,69,0,3.13,PY,1704414020
396,396,salalah,17.0151,54.0924,21.05,52,0,2.06,OM,1704414057
489,489,holualoa,19.6228,-155.9522,24.91,70,0,3.09,US,1704414069


### Step 3: Create a new DataFrame called `hotel_df`.

In [18]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_ideal_weather_df[['City','Country','Lat','Lng','Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']= " "


# Display sample data
hotel_df.head(10)

,City,Country,Lat,Lng,Humidity,Hotel Name
116,kailua-kona,US,19.6406,-155.9956,69,
164,san lorenzo,PY,-25.3333,-57.5333,69,
396,salalah,OM,17.0151,54.0924,52,
489,holualoa,US,19.6228,-155.9522,70,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [19]:
# Set parameters to search for a hotel
radius = 10000

params = {
    # YOUR CODE HERE
    "categories":'accommodation.hotel',
    "limit":20,
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
kailua-kona - nearest hotel: Kona Seaside Hotel
san lorenzo - nearest hotel: Hotel del Sur
salalah - nearest hotel: Muscat International Hotel
holualoa - nearest hotel: Kona Hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
116,kailua-kona,US,19.6406,-155.9956,69,Kona Seaside Hotel
164,san lorenzo,PY,-25.3333,-57.5333,69,Hotel del Sur
396,salalah,OM,17.0151,54.0924,52,Muscat International Hotel
489,holualoa,US,19.6228,-155.9522,70,Kona Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [20]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 2,
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)